# MongoDB via Mongo Shell

# Outline

- Opening the Mongo Shell
- Show Databases
- Import Data
- Select Imported DB
- Show Collections
- Sample Documents
- Get Record Counts
- Get List of Distinct Fields
- Get Number of Distinct Values by Field
- Get List of Unique Values for a Field
- Get Count of Unique Values by Value for a Field
- CRUD Operations
	- Delete and Create
	- Read
	- Update
- Indexes
	- View Indexes
	- Create Indexes

The following code is only valid with the MongoDB shell, wherever not otherwise stated.

# Opening the Mongo Shell

### From the Command Line

This will default to the localhost server.

In [ ]:
mongosh

In [ ]:
# to set the host and port:
# mongosh --host <hostname> --port <port>

### Opening it Directly

This will prompt for a server: 

<code>Please enter a MongoDB connection string (Default: mongodb://localhost/):</code>

# Show Databases

In [ ]:
show dbs

In [ ]:
# admin        132.00 KiB
# config       116.00 KiB
# local         96.00 KiB

<code>show databases</code> would work equally well.

# Import Data

### Drop <code>clickstream</code> if Exists (Optional)

In [ ]:
use clickstream

In [ ]:
# switched to db clickstream

In [ ]:
db.dropDatabase()

In [ ]:
# { ok: 1, dropped: 'clickstream' }

### Import From File

<code>mongorestore</code> and <code>mongoimport</code> must be run from the command line, not the mongo shell.

The following is stored in <code>import_data.bat</code>. <code>REM</code> is the comment indicator for <code>.bat</code> files.

In [ ]:
!"C:/Users/patwh/Documents/clickstream_data/import_data.bat"

In [ ]:
# 2025-05-28T18:38:13.827-0600	finished restoring clickstream.clicks (6100000 documents, 0 failures)
# 2025-05-28T18:38:13.827-0600	no indexes to restore for collection clickstream.clicks
# 2025-05-28T18:38:13.827-0600	6100000 document(s) restored successfully. 0 document(s) failed to restore.
# 2025-05-28T18:38:14.488-0600	connected to: mongodb://localhost:27017/
# 2025-05-28T18:38:14.490-0600	dropping: clickstream.clicks.metadata
# 2025-05-28T18:38:14.507-0600	1 document(s) imported successfully. 0 document(s) failed to import.

# Select Imported DB

In [ ]:
show dbs

In [ ]:
# admin        132.00 KiB
# clickstream  428.30 MiB
# config       108.00 KiB
# local         96.00 KiB

In [ ]:
use clickstream

In [ ]:
# switched to db clickstream

# Show Collections

In [ ]:
show collections

In [ ]:
# clicks
# clicks.metadata

# Sample Documents

In [ ]:
db.clicks.findOne()

In [ ]:
# {
#   _id: ObjectId('60df1029ad74d9467c91a932'),
#   webClientID: 'WI100000244987',
#   VisitDateTime: ISODate('2018-05-25T04:51:14.179Z'),
#   ProductID: 'Pr100037',
#   Activity: 'click',
#   device: { Browser: 'Firefox', OS: 'Windows' },
#   user: { City: 'Colombo', Country: 'Sri Lanka' }
# }

In [ ]:
db.clicks.metadata.findOne()

In [ ]:
# {
#   _id: ObjectId('6837ada071d28360c34516c3'),
#   indexes: [ { v: 2, key: { _id: 1 }, name: '_id_' } ],
#   uuid: 'ee6da5fe5bdf42b2bc3cecee40723af6',
#   collectionName: 'clicks'
# }

# Get Record Counts

In [ ]:
db.clicks.countDocuments()

In [ ]:
# 6100000

In [ ]:
db.clicks.metadata.countDocuments()

In [ ]:
# 1

### Count of Unique Values (Field <code>device.Browser</code>)

In [ ]:
db.clicks.distinct("device.Browser").length

In [ ]:
# 82

# Get List of Distinct Fields

Wrapping this in parentheses is necessary if typing it as a multi-line command (using Enter) in the Mongo Shell; otherwise it will execute the first line when Enter is hit.

(For multi-line commands, hit enter after each line. The shell will wait until shift+enter is used to execute.)

In [ ]:
# limiting to 1M records (it's faster with pymongo)

In [ ]:
(function() {
  var fields = {};
  db.clicks.find().limit(1000000).forEach(function(doc) {
    Object.keys(doc).forEach(function(key) {
      fields[key] = true;
    });
  });
  printjson(Object.keys(fields));
})();

In [ ]:
# [
#   '_id',
#   'webClientID',
#   'VisitDateTime',
#   'ProductID',
#   'Activity',
#   'device',
#   'user'
# ]

Including nested fields:

In [20]:
## Function for Creating .js Files From Text in Python

import os

def save_js_commands(js_input, js_folder, js_filename):
    filepath = f"{js_folder}/{js_filename}.js"
    
    try:
        # trim if string
        if isinstance(js_input, str):
            lines = [line.rstrip() for line in js_input.splitlines() if line.strip()]

        # trim each item if list
        elif isinstance(js_input, list):
            lines = [line.rstrip() for line in js_input if line.strip()]

        # raise error if not string or list
        else:
            raise TypeError("Input must be a string or list of strings.")

        # write to .js file
        with open(filepath, 'w', encoding='utf-8', newline='\r\n') as file:
            for line in lines:
                file.write(line + '\n')

        print(f"✅ JavaScript code saved successfully to: {filepath}")
        return filepath
    except Exception as e:
        print(f"❌ Error saving file: {e}")
        return None

In [14]:
JS_FOLDER = r"C:\Users\patwh\Documents\js_scripts"

In [23]:
## unique_fields_nested.js

js_code = """
(() => {
  let fields = {};

  // recursive function to extract fields
  function extractFields(obj, prefix = "") {

    // iterate through keys
    for (let key in obj) {

      // construct full key path, e.g. parent.child
      let fullKey = prefix + (prefix ? "." : "") + key;

      // store field
      fields[fullKey] = true;
      if (obj[key]?.constructor === Object) extractFields(obj[key], fullKey);
    }
  }

  // limit number of records searched
  db.clicks.find().limit(1000000).forEach(doc => extractFields(doc));

  // print results
  printjson(Object.keys(fields));
})();
"""

js_folder = JS_FOLDER
js_filename = "unique_fields_nested"

js_file = save_js_commands(js_code, js_folder, js_filename)

✅ JavaScript code saved successfully to: C:\Users\patwh\Documents\js_scripts/unique_fields_nested.js


In [ ]:
load("C:/Users/patwh/Documents/js_scripts/unique_fields_nested.js")

In [ ]:
# [
#   '_id',
#   'webClientID',
#   'VisitDateTime',
#   'ProductID',
#   'Activity',
#   'device',
#   'device.Browser',
#   'device.OS',
#   'user',
#   'user.City',
#   'user.Country',
#   'user.UserID'
# ]

# Get Number of Distinct Values by Field

Hard-coding the file names above (for speed):

In [16]:
## count_unique_value_hardcoded.js

js_code = """
(function() {
  const collection = db.clicks;

  const fields = [
    '_id',
    'webClientID',
    'VisitDateTime',
    'ProductID',
    'Activity',
    'device',
    'device.Browser',
    'device.OS',
    'user',
    'user.City',
    'user.Country',
    'user.UserID'
  ];

  // array to store fields and counts
  const results = [];

  // iterate through fields
  fields.forEach(field => {

    const pipeline = 
      // group documents by the specified field
      { $group: { _id: `$${field}` } },

      // count number of documents in group 
      { $group: { _id: null, count: { $sum: 1 } } }
    ];

    // run collection through pipeline, convert the result to array
    const result = collection.aggregate(pipeline).toArray();

    // if doc exists, extract count
    const count = result.length > 0 ? result[0].count : 0;
    results.push({ field: field, count: count });
  });

  // sort results descending by count
  results.sort((a, b) => b.count - a.count);

  // print sorted results
  results.forEach(({ field, count }) => {
    print(`${field}: ${count} unique values`);
  });
})();
"""

js_folder = JS_FOLDER
js_filename = "count_unique_values_hardcoded"

js_file = save_js_commands(js_code, js_folder, js_filename)

✅ JavaScript code saved successfully to: C:\Users\patwh\Documents\js_scripts/count_unique_values_hardcoded.js


In [5]:
load("C:/Users/patwh/Documents/js_scripts/count_unique_values_hardcoded.js")

In [ ]:
# _id: 6100000 unique values
# VisitDateTime: 6089023 unique values
# webClientID: 1091455 unique values
# user: 72162 unique values
# user.UserID: 34051 unique values
# user.City: 26260 unique values
# ProductID: 10938 unique values
# user.Country: 222 unique values
# device: 151 unique values
# device.Browser: 82 unique values
# device.OS: 18 unique values
# Activity: 2 unique values

### Dynamic Version (First Finds Fields, then Distinct Value Counts)

In [28]:
## count_unique_values_dynamic.js

js_code = """
(() => {
  db = db.getSiblingDB('clickstream');
  
  // empty object
  let fields = {};

  // extract fields from nested objects
  function extractFields(obj, prefix = '') {
  
    // iterate through keys
    Object.keys(obj).forEach(key => {
    
      // construct full key path, e.g. parent.child
      let fullKey = prefix + (prefix ? '.' : '') + key;
      
      // mark field as present
      fields[fullKey] = 1;
      
      // recurse into nested objects if valid
      if (obj[key]?.constructor === Object) extractFields(obj[key], fullKey);
    });
  }

  // iterate through documents in collection
  db.clicks.find().forEach(extractFields);

  // map fields to their unique value counts
  let results = Object.keys(fields).map(field => ({
  
    // store field name
    field,
    
    // count unique values
    count: db.clicks.aggregate([
    
      // group by field
      { $group: { _id: `$${field}` } },
      
      // count unique groups
      { $group: { _id: null, count: { $sum: 1 } } }
      
    // get first result's count or 0 if empty
    ]).toArray()[0]?.count || 0
    
  // sort by count in descending order
  })).sort((a, b) => b.count - a.count);

  // print ranked field counts
  results.forEach((r, i) => print(`${i + 1}. ${r.field}: ${r.count}`));
})();
"""

js_folder = JS_FOLDER
js_filename = "count_unique_values_dynamic"

js_file = save_js_commands(js_code, js_folder, js_filename)
exe_file = f"{js_folder}\\{js_filename}.js"

✅ JavaScript code saved successfully to: C:\Users\patwh\Documents\js_scripts/count_unique_values_dynamic.js


In [18]:
load("C:/Users/patwh/Documents/js_scripts/count_unique_values_dynamic.js")

In [ ]:
# === Starting Unique Field Count ===
# 1. _id: 6100000 unique values
# 2. VisitDateTime: 6089023 unique values
# 3. webClientID: 1091455 unique values
# 4. user: 72162 unique values
# 5. user.UserID: 34051 unique values
# 6. user.City: 26260 unique values
# 7. ProductID: 10938 unique values
# 8. user.Country: 222 unique values
# 9. device: 151 unique values
# 10. device.Browser: 82 unique values
# 11. device.OS: 18 unique values
# 12. Activity: 2 unique values
# === Done ===
# true

# Get List of Unique Values for a Field

### Get Unique Values for <code>device.Browser</code>

In [ ]:
db.clicks.distinct("device.Browser")

In [ ]:
# [
#   'AdsBot-Google',
#   'AhrefsBot',
#   'Amazon Silk',
#   'Android',
#   'AppEngine-Google',
#   'Apple Mail',
#   'BingPreview',
#   'BlackBerry WebKit',
#   'Chrome',
#   'Chrome Mobile',
#   'Chrome Mobile WebView',
#   'Chrome Mobile iOS',
#   'Chromium',
#   'Coc Coc',
#   'Coveobot',
#   'Crosswalk',
#   'Dragon',
#   'DuckDuckBot',
#   'Edge',
#   'Edge Mobile',
#   'Electron',
#   'Epiphany',
#   'Facebook',
#   'FacebookBot',
#   'Firefox',
#   'Firefox Mobile',
#   'Firefox iOS',
#   'HbbTV',
#   'HeadlessChrome',
#   'HubSpot Crawler',
#   'IE',
#   'IE Mobile',
#   'Iceweasel',
#   'Iron',
#   'JobBot',
#   'Jooblebot',
#   'K-Meleon',
#   'Kindle',
#   'Konqueror',
#   'Magus Bot',
#   'Mail.ru Chromium Browser',
#   'Maxthon',
#   'Mobile Safari',
#   'Mobile Safari UI/WKWebView',
#   'MobileIron',
#   'NetFront',
#   'Netscape',
#   'Opera',
#   'Opera Coast',
#   'Opera Mini',
#   'Opera Mobile',
#   'Other',
#   'PagePeeker',
#   'Pale Moon',
#   'PetalBot',
#   'PhantomJS',
#   'Pinterest',
#   'Puffin',
#   'Python Requests',
#   'QQ Browser',
#   'QQ Browser Mobile',
#   'Radius Compliance Bot',
#   'Safari',
#   'Samsung Internet',
#   'SeaMonkey',
#   'Seekport Crawler',
#   'SiteScoreBot',
#   'Sleipnir',
#   'Sogou Explorer',
#   'Thunderbird',
#   'UC Browser',
#   'Vivaldi',
#   'WebKit Nightly',
#   'WordPress',
#   'Yandex Browser',
#   'YandexAccessibilityBot',
#   'YandexBot',
#   'YandexSearch',
#   'Yeti',
#   'YisouSpider',
#   'moatbot',
#   'net/bot'
# ]

# Get Count of Unique Values for a Field

In [29]:
# count_unique_values_for_field.js

js_code = """
(function() {
  // get clicks collection
  const collection = db.getSiblingDB('clickstream').clicks;
  
  // define aggregation pipeline
  const pipeline = [
  
    // group by Browser, count occurrences
    { $group: { _id: "$device.Browser", count: { $sum: 1 } } },
    
    // sort by count descending
    { $sort: { count: -1 } }
  ];
  
  // run pipeline, get results
  const result = collection.aggregate(pipeline).toArray();
  
  // print each browser and count
  result.forEach(doc => {
    print(`${doc._id}: ${doc.count}`);
  });
})();
"""

js_folder = JS_FOLDER
js_filename = "count_unique_values_for_field"

js_file = save_js_commands(js_code, js_folder, js_filename)

✅ JavaScript code saved successfully to: C:\Users\patwh\Documents\js_scripts/count_unique_values_for_field.js


In [6]:
load("C:/Users/patwh/Documents/js_scripts/count_unique_values_for_field.js")

In [ ]:
# Chrome: 4360498
# Chrome Mobile: 788991
# Firefox: 388766
# Safari: 204343
# Mobile Safari: 93241
# HeadlessChrome: 76595
# Opera: 62522
# Chrome Mobile iOS: 28031
# Chrome Mobile WebView: 25381
# Samsung Internet: 19804
# Firefox Mobile: 7040
# Yandex Browser: 6265
# Facebook: 5874
# Opera Mobile: 5509
# Edge: 5180
# Coc Coc: 4506
# IE: 3381
# Mobile Safari UI/WKWebView: 3038
# Apple Mail: 3038
# UC Browser: 2142
# Firefox iOS: 884
# Chromium: 629
# QQ Browser: 498
# Jooblebot: 466
# PhantomJS: 440
# Amazon Silk: 376
# AdsBot-Google: 312
# Pinterest: 288
# AhrefsBot: 229
# moatbot: 216
# JobBot: 208
# Sogou Explorer: 200
# Iron: 156
# Maxthon: 139
# Dragon: 105
# Pale Moon: 95
# PetalBot: 82
# Other: 58
# YandexBot: 56
# Vivaldi: 43
# YandexAccessibilityBot: 41
# BingPreview: 33
# Crosswalk: 27
# Thunderbird: 23
# IE Mobile: 22
# AppEngine-Google: 22
# Android: 19
# Edge Mobile: 17
# Puffin: 16
# QQ Browser Mobile: 15
# Epiphany: 15
# Electron: 13
# FacebookBot: 12
# Opera Mini: 12
# Yeti: 9
# WordPress: 9
# Coveobot: 8
# PagePeeker: 6
# Sleipnir: 6
# SeaMonkey: 5
# Mail.ru Chromium Browser: 5
# SiteScoreBot: 4
# HbbTV: 4
# Seekport Crawler: 3
# MobileIron: 3
# Kindle: 3
# Netscape: 2
# Python Requests: 2
# Iceweasel: 2
# BlackBerry WebKit: 2
# K-Meleon: 2
# Radius Compliance Bot: 2
# Opera Coast: 2
# HubSpot Crawler: 1
# NetFront: 1
# DuckDuckBot: 1
# WebKit Nightly: 1
# net/bot: 1
# Konqueror: 1
# YisouSpider: 1
# YandexSearch: 1
# Magus Bot: 1

# CRUD Operations

## Delete and Create

### Delete the Last Record

Remove the last record from <code>clicks</code> and re-insert it.

First capture in variable:

In [ ]:
var lastDoc = db.clicks.find().sort({ _id: -1 }).limit(1).next();

In [ ]:
# {
#   _id: ObjectId('60df129dad74d9467ceebd51'),
#   webClientID: 'WI100000118333',
#   VisitDateTime: ISODate('2018-05-26T11:51:44.263Z'),
#   ProductID: 'Pr101251',
#   Activity: 'click',
#   device: { Browser: 'Chrome', OS: 'Windows' },
#   user: { City: 'Vijayawada', Country: 'India' }
# }

Delete it:

In [ ]:
db.clicks.deleteOne({ _id: lastDoc._id });

In [ ]:
# { acknowledged: true, deletedCount: 1 }

### Insert it Back In

In [ ]:
db.clicks.insertOne(lastDoc)

In [ ]:
# {
#   acknowledged: true,
#   insertedId: ObjectId('60df129dad74d9467ceebd51')
# }

### Delete the Last 5 Records

In [ ]:
var lastDocs = db.clicks.find().sort({ _id: -1 }).limit(5).toArray();
var idsToDelete = lastDocs.map(doc => doc._id);

In [ ]:
db.clicks.deleteMany({ _id: { $in: idsToDelete } });

In [ ]:
# { acknowledged: true, deletedCount: 5 }

### Insert Them Back In

In [ ]:
db.clicks.insertMany(lastDocs);

In [ ]:
# {
#   acknowledged: true,
#   insertedIds: {
#     '0': ObjectId('60df129dad74d9467ceebd51'),
#     '1': ObjectId('60df129dad74d9467ceebd50'),
#     '2': ObjectId('60df129dad74d9467ceebd4f'),
#     '3': ObjectId('60df129dad74d9467ceebd4e'),
#     '4': ObjectId('60df129dad74d9467ceebd4d')
#   }
# }

## Read

### Filter to <code>_id</code> Equal to <code>60df129dad74d9467ceebd51</code>

In [ ]:
db.clicks.findOne({ _id: ObjectId("60df129dad74d9467ceebd51") });

In [ ]:
# {
#   _id: ObjectId('60df129dad74d9467ceebd51'),
#   webClientID: 'WI100000118333',
#   VisitDateTime: ISODate('2018-05-26T11:51:44.263Z'),
#   ProductID: 'Pr101251',
#   Activity: 'click',
#   device: { Browser: 'Chrome', OS: 'Windows' },
#   user: { City: 'Vijayawada', Country: 'India' }
# }

### Find First Record Where <code>device.Browser</code> is not Firefox

In [ ]:
db.clicks.findOne({ "device.Browser": "Firefox" });

In [ ]:
# {
#   _id: ObjectId('60df1029ad74d9467c91a932'),
#   webClientID: 'WI100000244987',
#   VisitDateTime: ISODate('2018-05-25T04:51:14.179Z'),
#   ProductID: 'Pr100037',
#   Activity: 'click',
#   device: { Browser: 'Firefox', OS: 'Windows' },
#   user: { City: 'Colombo', Country: 'Sri Lanka' }
# }

### Find First 2 Records Where <code>device.Browser</code> is not Firefox

In [ ]:
db.clicks.find({ "device.Browser": { $ne: "Firefox" } }).limit(2);

In [ ]:
# [
#   {
#     _id: ObjectId('60df1029ad74d9467c91a933'),
#     webClientID: 'WI10000061461',
#     VisitDateTime: ISODate('2018-05-25T05:06:03.700Z'),
#     ProductID: 'Pr100872',
#     Activity: 'pageload',
#     device: { Browser: 'Chrome Mobile', OS: 'Android' },
#     user: {}
#   },
#   {
#     _id: ObjectId('60df1029ad74d9467c91a934'),
#     webClientID: 'WI10000075748',
#     VisitDateTime: ISODate('2018-05-17T11:51:09.265Z'),
#     ProductID: 'Pr100457',
#     Activity: 'click',
#     device: { Browser: 'Chrome', OS: 'Linux' },
#     user: { City: 'Ottawa', Country: 'Canada' }
#   }
# ]

### Find First 2 Records Where <code>VisitDateTime</code> > 5/20/2018

In [ ]:
db.clicks.find({
  VisitDateTime: { $gt: new Date("2018-05-20") }
}).limit(2);

In [ ]:
# [
#   {
#     _id: ObjectId('60df1029ad74d9467c91a932'),
#     webClientID: 'WI100000244987',
#     VisitDateTime: ISODate('2018-05-25T04:51:14.179Z'),
#     ProductID: 'Pr100037',
#     Activity: 'click',
#     device: { Browser: 'Firefox', OS: 'Windows' },
#     user: { City: 'Colombo', Country: 'Sri Lanka' }
#   },
#   {
#     _id: ObjectId('60df1029ad74d9467c91a933'),
#     webClientID: 'WI10000061461',
#     VisitDateTime: ISODate('2018-05-25T05:06:03.700Z'),
#     ProductID: 'Pr100872',
#     Activity: 'pageload',
#     device: { Browser: 'Chrome Mobile', OS: 'Android' },
#     user: {}
#   }
# ]

### Get the Minimum and Maximum <code>VisitDateTime</code>

In [ ]:
db.clicks.aggregate([
  { $group: {
      _id: null,
      minVisitDateTime: { $min: "$VisitDateTime" },
      maxVisitDateTime: { $max: "$VisitDateTime" }
    } }
]);

In [ ]:
# [
#   {
#     _id: null,
#     minVisitDateTime: ISODate('2018-05-07T00:00:01.190Z'),
#     maxVisitDateTime: ISODate('2018-05-27T23:59:59.576Z')
#   }
# ]

### Get Count of Records Where <code>VisitDateTime</code> is Greater Than 5/20/2018

In [ ]:
db.clicks.countDocuments({
  VisitDateTime: { $gt: new Date("2018-05-20") }
});

In [ ]:
# 2453050

### Get Count of Records Where <code>user.Country</code> is <code>India</code> or <code>United States</code>

#### Using <code>$or</code>

In [ ]:
db.clicks.countDocuments({
  $or: [
    { "user.Country": "India" },
    { "user.Country": "United States" }
  ]
});

In [ ]:
# 3497232

#### Using <code>$in</code>

In [ ]:
db.clicks.countDocuments({
  "user.Country": { $in: ["India", "United States"] }
});

In [ ]:
# 3497232

### Get Count of Records Where <code>user.Country</code> is Neither <code>India</code> Nor <code>United States</code>

#### Using <code>$and</code>

In [ ]:
db.clicks.countDocuments({
  $and: [
    { "user.Country": { $ne: "India" } },
    { "user.Country": { $ne: "United States" } }
  ]
});

In [ ]:
# 2602768

#### Using `$not` and `$in`

In [ ]:
db.clicks.countDocuments({
  "user.Country": { $not: { $in: ["India", "United States"] } }
});

In [ ]:
# 2602768

#### Using <code>$nin</code>

In [ ]:
db.clicks.countDocuments({
  "user.Country": { $nin: ["India", "United States"] }
});

In [ ]:
# 2602768

### Get Count of Records with <code>user.UserID</code>

In [ ]:
db.clicks.countDocuments({
  "user.UserID": { $exists: true, $ne: null }
});

In [ ]:
# 602293

## Update

### Update <code>device.Browser</code> for Record <code>60df129dad74d9467ceebd51</code> to <code>Firefox</code>

In [ ]:
db.clicks.updateOne(
  { _id: ObjectId("60df129dad74d9467ceebd51") },
  { $set: { "device.Browser": "Firefox" } }
);

In [ ]:
# {
  # acknowledged: true,
  # insertedId: null,
  # matchedCount: 1,
  # modifiedCount: 1,
  # upsertedCount: 0
# }

Set it back to original state for accuracy:

In [ ]:
db.clicks.updateOne(
  { _id: ObjectId("60df129dad74d9467ceebd51") },
  { $set: { "device.Browser": "Chrome" } }
);

In [ ]:
# {
  # acknowledged: true,
  # insertedId: null,
  # matchedCount: 1,
  # modifiedCount: 1,
  # upsertedCount: 0
# }

### Update <code>device.Browser</code> Records to be <code>Firefox</code> if Set to <code>Firefox iOS</code>

In [ ]:
# db.clicks.updateMany(
#   { "device.Browser": "Firefox iOS" },
#   { $set: { "device.Browser": "Firefox" } }
# );

## Create New Field

### Add Field Called <code>NewField</code> to First 1000 Records, Set Value to <code>Default</code>

In [ ]:
db.clicks.find().limit(1000).forEach(doc => {
  db.clicks.updateOne(
    { _id: doc._id },
    { $set: { NewField: "Default" } }
  );
});

In [ ]:
db.clicks.findOne({ NewField: "Default" });

In [ ]:
# {
#   _id: ObjectId('60df1029ad74d9467c91a932'),
#   webClientID: 'WI100000244987',
#   VisitDateTime: ISODate('2018-05-25T04:51:14.179Z'),
#   ProductID: 'Pr100037',
#   Activity: 'click',
#   device: { Browser: 'Firefox', OS: 'Windows' },
#   user: { City: 'Colombo', Country: 'Sri Lanka' },
#   NewField: 'Default'
# }

### Remove the Added Field

In [ ]:
db.clicks.updateMany(
  { NewField: { $exists: true } },
  { $unset: { NewField: "" } }
);

In [ ]:
# {
#   acknowledged: true,
#   insertedId: null,
#   matchedCount: 1000,
#   modifiedCount: 1000,
#   upsertedCount: 0
# }

# Indexes

## View Indexes

In [ ]:
db.clicks.getIndexes()

In [ ]:
# [ { v: 2, key: { _id: 1 }, name: '_id_' } ]

In [ ]:
db.clicks.metadata.findOne();

In [ ]:
# {
#   _id: ObjectId('6837ada071d28360c34516c3'),
#   indexes: [ { v: 2, key: { _id: 1 }, name: '_id_' } ],
#   uuid: 'ee6da5fe5bdf42b2bc3cecee40723af6',
#   collectionName: 'clicks'
# }

## Create Indexes

In [1]:
db.clicks.createIndex({ "device.OS": 1 })

device.OS_1


In [ ]:
# device.OS_1

In [ ]:
db.clicks.createIndex({ "device.Browser": 1 })

In [ ]:
# device.Browser_1